In [1]:
import pandas as pd

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import keys

In [4]:
def delete_people_without_all_information(p):
    global subjects_processed

    p_data = subjects_processed[(subjects_processed['cod_plan'] == p.cod_plan) &
                               (subjects_processed['expediente'] == p.expediente)]
    if len(p_data[(p_data['tipologia_asignatura'] == 'T')
                  & (p_data['curso_asignatura'] == 1)
                  & (p_data['numero_convocatorias_agotadas'] <= 1)].index) < 10:
        return False
    else:
        return True

In [5]:
path_personal_data = '../../../Data/Pre-processed/personal_data.csv'
sep = '|'

In [6]:
personal_data = pd.read_csv(
                path_personal_data,
                sep=sep
                )
personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,exp_cerrado,exp_trasladado,tipo_traslado,exp_bloqueado,abandona,anio_convocatoria_acceso,...,des_subacesso,nota_acceso,nota_admision_def,centro_escolar_acceso,sexo,fecha_nacimiento,cod_provincia,provincia,cod_municipio,municipio
0,2,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,2007-08,S,N,NaN,N,N,2004-05,...,.,NaN,NaN,NaN,H,1981-10-23,10.0,CÁCERES,1.0,CÁCERES
1,3,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,2007-08,S,N,NaN,N,N,2000-01,...,.,NaN,NaN,NaN,H,1977-07-09,10.0,CÁCERES,1.0,CÁCERES
2,5,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,2007-08,S,N,NaN,N,N,1994-95,...,.,NaN,NaN,NaN,D,1970-08-26,6.0,BADAJOZ,345.0,JEREZ DE LOS CABALLEROS
3,6,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,2007-08,N,N,NaN,N,S,2003-04,...,.,NaN,NaN,NaN,H,1978-11-16,6.0,BADAJOZ,740.0,VILLAFRANCA DE LOS BARROS
4,7,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,2007-08,S,N,NaN,N,N,1996-97,...,.,NaN,NaN,NaN,H,1971-11-01,6.0,BADAJOZ,1.0,BADAJOZ


In [7]:
global subjects_processed
path_subjects_processed ='../../../Data/Processed/pr_subjects.csv'
sep = '|'
subjects_processed = pd.read_csv(
            path_subjects_processed,
            sep=sep
            )
subjects_processed.head()

,expediente,cod_plan,des_plan,cod_asignatura,des_asignatura,curso_asignatura,semestre,tipo_linea,tipologia_asignatura,des_tipogia_asignatura,curso_academico,convocatoria,nota_alf,nota_num,des_nota,numero_convocatorias_agotadas,fin_estudios
0,2,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,400103,PROCESAMIENTO DE IMÁGENES Y REUTILIZACIÓN DEL ...,1.0,A,O,B,OBLIGATORIA,2007-08,JUN,SB,9.5,SOBRESALIENTE,1,S
1,2,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,400105,COMPUTACIÓN RECONFIGURABLE,1.0,A,O,B,OBLIGATORIA,2007-08,JUN,SB,9.0,SOBRESALIENTE,1,S
2,2,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,400106,SISTEMAS EMPOTRADOS DISTRIBUIDOS,1.0,A,O,B,OBLIGATORIA,2007-08,JUN,NT,8.8,NOTABLE,1,S
3,2,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,400107,FUNDAMENTOS DE LA COMPUTACIÓN GRID Y PROYECTOS...,1.0,A,O,B,OBLIGATORIA,2007-08,JUN,SB,9.0,SOBRESALIENTE,1,S
4,2,1621,MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO,400109,COMPUTACIÓN CON CLUSTERS,1.0,A,O,B,OBLIGATORIA,2007-08,JUN,NT,8.0,NOTABLE,1,S


In [8]:
old_degrees = ['MÁSTER UNIVERSITARIO EN COMPUTACIÓN GRID Y PARALELISMO',
           'MÁSTER DE ESPECIALIZACIÓN EN GEOTECNOLOGÍAS TOPOGRÁFICAS EN LA INGENIERÍA',
           'MÁSTER UNIVERSITARIO EN EVALUACIÓN Y GESTIÓN DEL RUIDO AMBIENTAL',
           'MÁSTER EN COMPUTACIÓN GRID Y PARALELISMO']
no_valid_courses = ['2017-18', '2018-19', '2019-20', '2020-21']


In [9]:
print("initial number of rows: " + str(len(personal_data.index)))
rows_before = len(personal_data.index)
personal_data = personal_data[personal_data[keys.PLAN_DESCRIPTION_KEY].isin(old_degrees) == False]
rows_after = len(personal_data.index)
print("delete data of old degrees = " + str(rows_before - rows_after))

initial number of rows: 5426
delete data of old degrees = 97


In [10]:
rows_before = len(personal_data.index)
df_filter = personal_data.apply(delete_people_without_all_information, axis=1)
personal_data = personal_data[df_filter]
personal_data = personal_data[personal_data[keys.OPEN_YEAR_PLAN_KEY].isin(no_valid_courses)
                                      == False]
rows_after = len(personal_data.index)
print("delete data of people without all information = " + str( rows_before - rows_after))

delete data of people without all information = 2962


In [11]:
rows_affected = len(personal_data[pd.isna(personal_data[keys.TRANSFER_TYPE_KEY])].index)
personal_data[keys.TRANSFER_TYPE_KEY] = personal_data[keys.TRANSFER_TYPE_KEY].apply(
    lambda func: 'N' if pd.isna(func) else func)
print("resolve values null of tipo_tralado column = " + str( rows_affected))

resolve values null of tipo_tralado column = 1689


In [12]:
print("columns of final dataset are:" + str(personal_data.columns))
print("final number of rows: " + str(len(personal_data.index)))

columns of final dataset are:Index(['expediente', 'cod_plan', 'des_plan', 'anio_apertura_expediente',
       'exp_cerrado', 'exp_trasladado', 'tipo_traslado', 'exp_bloqueado',
       'abandona', 'anio_convocatoria_acceso', 'convocatoria_acceso', 'acceso',
       'des_acceso', 'sub_acceso', 'des_subacesso', 'nota_acceso',
       'nota_admision_def', 'centro_escolar_acceso', 'sexo',
       'fecha_nacimiento', 'cod_provincia', 'provincia', 'cod_municipio',
       'municipio'],
      dtype='object')
final number of rows: 2367


In [13]:
path_personal_data_processed ='../../../Data/Processed/pr_personal_data.csv'
personal_data.to_csv(
    path_personal_data_processed,
    sep=sep,
    index=False)